In [1]:
import pandas as pd

# Leitura de dados

* ```parse_dates``` em ```pd.read_csv(parse_dates =)```

Na documentação, está o seguinte:

*"```parse_dates``` : boolean or list of ints or names or list of lists or dict, default False*

***boolean. If True -> try parsing the index.***
*[...]*
*If a column or index contains an unparseable date, the entire column or index will be returned unaltered as an object data type. For non-standard datetime parsing, use pd.to_datetime after pd.read_csv*

*Note: **A fast-path exists for iso8601-formatted dates.**"*

Ou seja, basicamente, ele já transforma as datas em ```datetime``` para você, o que pode ajudar muito a vida. É mais fácil isso acontecer se as strings das datas estiverem no formato datetime.

**Provavelmente é muito importante para mexer com time series.**

Obs.: ler mais sobre o parse_dates em https://stackoverflow.com/questions/17465045/can-pandas-automatically-read-dates-from-a-csv-file

* Jeitos de fazer a visualização inicial dos dados:

```df```

```df.head(n)```

```df.tail(n)```

```df.sample(n)```

# Limpeza de dados

### Lidando com NaN values

* Descobrindo se há valores NaN


# Manipulação de dados

* ```df.rename```

Podemos renomear o nome de um index usando o típico ```df.rename```. Note que fazer

```df.rename(mapper, axis = 0)``` é o mesmo que ```df.rename(index = mapper)``` e ```df.rename(mapper, axis=  1)``` é o mesmo que ```df.rename(columns = mapper)```

## Index e Multiindex

* Definindo os index (se for colocada mais de uma coluna, será um multiindex)

```df = df.set_index(['Date', 'Store', 'Category', 'Subcategory', 'Description'])```

* Fazendo ```append``` de um novo index aos index que já existe

```df = df.set_index('UPC EAN', append = True)```

* Definindo a ordem dos multiindex

```df = df.reorder_levels(order = ['Date', 'Store', 'Category', 'Subcategory', 'UPC EAN', 'Description'])```

* Trocando a ordem de dois *```levels```* de um multiindex:

df.swaplevel()

* Mudando o nome de um level de um multiindex


```df.index = df.index.set_names('Desc.', level = 'Description') # especificamos o novo nome do level e também de qual level estamos tratando```

* Função que 'alisa' multicolumns e transforma as tuplas das multicolumns em colunas únicas:


In [ ]:
'''
def flatten_cols(df: pd.DataFrame, delim: str = ""):
    """Flatten multiple column levels of the DataFrame into a one column level.

    Args:
        delim: the delimiter between the column values.

    Returns:
        A copy of the dataframe with the new column names.

    """
    new_cols = [delim.join(col_lev for col_lev in tup if col_lev) for tup in df.columns.values] 
    ndf = df.copy()
    ndf.columns = new_cols # Redefinimos o que são as colunas
    return ndf

flattened_multi_col_df = flatten_cols(multi_col_lvl_df, " | ")
flattened_multi_col_df.head(3)
'''

* Função que faz o procedimento contrário: pega colunas 'alisadas' e as transforma em multicolumns

In [ ]:
'''
def unflatten_cols(df: pd.DataFrame, delim: str = ""):
    """Unflatten a single column level into multiple column levels.

    Args:
        delim: the delimiter to split on to identify the multiple column values.

    Returns:
        A copy of the dataframe with the new column levels.

    """
    new_cols = pd.MultiIndex.from_tuples([tuple(col.split(delim)) for col in df.columns]) 
    ndf = df.copy()
    ndf.columns = new_cols
    return ndf

unflatten_cols(flattened_multi_col_df, " | ").head(3)
'''

* Tirando um index e transformando ele em um level do column index: 

```df.unstack('index_label')```

* Tirando um level do columnindex e transformando em um index:

```df.stack('index_label')```

Obs.: a documentação de ```stack()``` é intuitiva:
"*Stack the prescribed level(s) from columns to index.*"

* Transformando um df com multicolumns e multiindex em um df normal

def multi_index_to_normal_df(df: pd.DataFrame):
    """
    Pega um df com multiindex e multicolumn e 'alisa' o index e as colunas.
    # a) 'restacka' os levels das colunas e joga eles para os indices.
    # b) dropa qualquer espaço vazio que o stacking deixou,
    # c) reseta o index para o multiindex virar várias colunas denovo.
    """
    normal_df = df.copy()
    return (normal_df.stack().dropna().reset_index())

## Datetimes

* Referenciar datetimes em um dataframe

Quando os dados estão em ```datetime```, nós podemos referenciá-los simplesmente a partir de uma string. Ex.:

```df.type()```

    #   Column       Non-Null Count  Dtype   

    ---  ------       --------------  -----        

    0   Date         132 non-null    datetime64[ns]
    

```df[df['Date'] == '2018-07-10']```

## Slicing

* Fazer ```label slicing``` dentro do ```.loc[]``` com *mais de um intervalo de colunas*. Fonte: https://stackoverflow.com/questions/72246028/pandas-specify-slice-additional-column-label-in-loc

Ex.: digamos que as colunas de um dataset são:
```Year | Country Name | ER | RR | BBR | DR```

Você quer selecionar somente as colunas ```Year``` e as colunas de ```ER``` *até* ```DR```.

Uma forma de fazer esse slicing é:

```df.loc[:, ['Year', *filt_df.loc[:, 'ER':'DR'].columns]]``` 

Explicação: usamos o ```.loc[]``` duas vezes. O ```*``` acessa o nome das colunas que estão em ```.columns```.



# Plotagem de dados

# Exportação de dados

* Exportar dataframe para excel ou para csv: ```pd.to_excel()``` ou ```pd.to_csv()```

```df.to_excel('file_name', sheet = 'sheet_name', index = False)```

* Para exportar várias tabelas para um mesmo arquivo excel:

```with pd.ExcelWriter("output.xlsx") as writer:```

      ```df1(writer, sheet_name="name_df1", index = True)```

      ```df2(writer, sheet_name="name_df2", index = True)```
   
      ```df3(writer, sheet_name="name_df3", index = True)```


Obs.: usar ```\\``` ao invés de só ```\``` nas strings dos ```paths```